# Copernicus API

---

Before using the code below, you need to create an [account](https://dataspace.copernicus.eu/). If you have already an account, replace your id and password in the def _init_ inside the class Copernicus_API below:

---

## Import

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
#pip install jinja2     Mandatory for the dataframe style

---

## Class creation

In [ ]:
class Copernicus_API:
    """
    A class for interacting with the Copernicus Dataspace Ecosystem (CDSE) API.

    This class handles:
    - Authentication with the CDSE API using a token.
    - Searching for available satellite products using OData queries.
    - Displaying search results in a paginated table.
    - Displaying quicklook thumbnails interactively.
    - Downloading selected products with progress feedback.

    Attributes:
        username (str): The username for authentication.
        password (str): The password for authentication.
        token (str): Bearer token used for authenticated API requests.
        products (list): List of product metadata returned from a search.
        df (pd.DataFrame): DataFrame version of the product list.
    """
#################################################################################################################################
    def __init__(self):
        """
        Initialize the Copernicus_API instance.

        Loads environment variables for CDSE authentication credentials,
        authenticates to obtain an access token, and initializes
        product storage attributes.

        Attributes initialized:
            self.username (str): Username loaded from environment variable 'CDSE_USERNAME'.
            self.password (str): Password loaded from environment variable 'CDSE_PASSWORD'.
            self.token (str): Access token obtained after authentication.
            self.products (list): List to store product metadata dictionaries.
            self.df (pd.DataFrame or None): DataFrame to store product search results.

        Raises:
            Exception: If authentication fails during token retrieval.
        """
        self.username = "remotesensing@gmail.com" # Your id
        self.password = "Iloveremotesensing" # Your password
        self.token = self._authenticate()
        self.products = []
        self.df = None
#################################################################################################################################
    def _authenticate(self):
        """
        Authenticate the user and obtain an access token from the CDSE identity service.

        Sends a POST request with the username and password to the authentication endpoint
        and retrieves an OAuth2 access token.

        Returns:
            str: Access token string used for authorization in subsequent API requests.

        Raises:
            Exception: If authentication fails (non-200 HTTP status), an exception
            is raised with the status code and error message.
        """
        url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "username": self.username,
            "password": self.password,
            "grant_type": "password",
            "client_id": "cdse-public"
        }
        response = requests.post(url, headers=headers, data=data)
        if response.status_code == 200:
            print(" Token generated")
            return response.json()["access_token"]
        else:
            raise Exception(f"Authentication failed: {response.status_code}\n{response.text}")
#################################################################################################################################    
    def refresh_token(self):
        """
        Refresh the current authentication token by re-authenticating.

        Calls the internal `_authenticate` method to get a new access token and updates
        the `self.token` attribute.

        Returns:
            str: The refreshed access token.
        """
        self.token = self._authenticate()
        return self.token
#################################################################################################################################
    def search_products(self, params):
        """
        Search for products in the Copernicus Dataspace Catalogue using given query parameters.

        Sends a GET request to the catalogue API with the specified search parameters.
        Parses the response JSON to extract relevant product information, including
        product ID, name, acquisition start date, content length in MB, and Quicklook asset ID.
        Stores the product list internally and creates a pandas DataFrame `self.df` for convenience.

        Args:
            params (dict): Dictionary of query parameters to filter the search.

        Raises:
            Exception: If the HTTP request fails or returns a non-200 status code.
        """
        url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
        headers = {"Authorization": f"Bearer {self.token}"}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            results = response.json().get("value", [])
            self.products = []
            for product in results:
                product_id = product.get("Id")
                product_name = product.get("Name")
                content_date = product.get("ContentDate", {})
                content_length = round(product.get("ContentLength", 0) / 1e6, 2)
                assets = product.get("Assets", [])

                quicklook_asset_id = None
                for asset in assets:
                    if asset.get("Type") == "QUICKLOOK":
                        quicklook_asset_id = asset.get("Id")
                        break

                self.products.append({
                    "Product ID": product_id,
                    "Name": product_name,
                    "Acquisition Start": content_date.get("Start"),
                    "Content Length (MB)": content_length,
                    "Quicklook Asset ID": quicklook_asset_id
                })

            self.df = pd.DataFrame(self.products)
        else:
            raise Exception(f"Search failed: {response.status_code}\n{response.text}")
#################################################################################################################################
    def display_df(self):
        """
        Display the list of products in a paginated interactive table within a Jupyter notebook.

        Shows 10 products per page with navigation controls:
        - Buttons for first page, previous page, next page, and last page.
        - A bounded integer input to enter and jump to a specific page.
        - A label showing the total number of pages.

        The product DataFrame excludes the 'Quicklook Asset ID' column for cleaner display.
        The table styling centers the text for readability.

        If no products are available, a message is printed instead.

        No parameters or return values.
        """
        if not self.products:
            print("No products to display.")
            return

        per_page = 10
        total = len(self.products)
        total_pages = (total - 1) // per_page + 1

        table_output = widgets.Output()
        controls_output = widgets.Output()

        # Navigation widgets
        current_page = widgets.BoundedIntText(value=1, min=1, max=total_pages, layout=widgets.Layout(width="60px"))
        max_label = widgets.Label(f"/ {total_pages}", layout=widgets.Layout(padding="0 10px"))

        first_button = widgets.Button(description="⏮", layout=widgets.Layout(width="35px"))
        left_button = widgets.Button(description="←", layout=widgets.Layout(width="35px"))
        right_button = widgets.Button(description="→", layout=widgets.Layout(width="35px"))
        last_button = widgets.Button(description="⏭", layout=widgets.Layout(width="35px"))

        def update_table(page):
            with table_output:
                clear_output(wait=True)
                start = (page - 1) * per_page
                end = min(start + per_page, total)
                sub_df = pd.DataFrame(self.products[start:end]).drop(columns=["Quicklook Asset ID"], errors="ignore")
                sub_df.index = range(start, end)

                styled_df = sub_df.style.set_properties(**{
                    'text-align': 'center',
                    'white-space': 'normal'
                }).set_table_styles([
                    {'selector': 'th', 'props': [('text-align', 'center')]},
                    {'selector': 'td', 'props': [('text-align', 'center')]}
                ])

                display(styled_df)

            with controls_output:
                clear_output(wait=True)
                nav_controls = widgets.HBox(
                    [first_button, left_button, current_page, max_label, right_button, last_button],
                    layout=widgets.Layout(justify_content="center", align_items="center", gap="5px")
                )
                display(nav_controls)

        # Navigation logic
        def go_first(_): current_page.value = 1
        def go_left(_):  current_page.value = max(1, current_page.value - 1)
        def go_right(_): current_page.value = min(total_pages, current_page.value + 1)
        def go_last(_):  current_page.value = total_pages

        def on_page_change(change):
            update_table(change["new"])

        # Bind actions
        first_button.on_click(go_first)
        left_button.on_click(go_left)
        right_button.on_click(go_right)
        last_button.on_click(go_last)
        current_page.observe(on_page_change, names="value")

        display(table_output)
        display(controls_output)
        update_table(1)
#################################################################################################################################
    def quicklooks(self):
        """
        Display an interactive widget to browse quicklook images of the current products.

        Features:
        - Shows one quicklook image at a time with the product name as a title.
        - Includes navigation controls: first, previous, next, and last buttons.
        - Allows manual input of image index via a bounded integer widget.
        - Includes a play button to automatically cycle through images every second.
        - Automatically refreshes the authentication token if it expires during image retrieval.
        - Displays error messages if an image cannot be fetched.

        If no products or no quicklook images are available, prints an informative message.

        No parameters or return values.
        """
        if not self.products:
            print("No products to display.")
            return

        quicklook_items = [(prod["Name"], prod["Quicklook Asset ID"]) for prod in self.products if prod.get("Quicklook Asset ID")]

        if not quicklook_items:
            print("No quicklook images available.")
            return

        max_index = len(quicklook_items) - 1
        output = widgets.Output()

        # Controls
        play = widgets.Play(
            value=0,
            min=0,
            max=max_index,
            step=1,
            interval=1000,
            description="▶️",
            disabled=False
        )

        current_index = widgets.BoundedIntText(
            value=0,
            min=0,
            max=max_index,
            layout=widgets.Layout(width="60px")
        )

        max_label = widgets.Label(f"/ {max_index}")

        # Navigation buttons
        first_button = widgets.Button(description="⏮", layout=widgets.Layout(width="35px"))
        left_button = widgets.Button(description="←", layout=widgets.Layout(width="35px"))
        right_button = widgets.Button(description="→", layout=widgets.Layout(width="35px"))
        last_button = widgets.Button(description="⏭", layout=widgets.Layout(width="35px"))

        widgets.jslink((play, 'value'), (current_index, 'value'))

        def update_display(index):
            name, asset_id = quicklook_items[index]
            url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Assets({asset_id})/$value"
            headers = {"Authorization": f"Bearer {self.token}"}
            response = requests.get(url, headers=headers)

            # Refresh token if expired
            if response.status_code == 401:
                self.refresh_token()
                headers = {"Authorization": f"Bearer {self.token}"}
                response = requests.get(url, headers=headers)

            with output:
                clear_output(wait=True)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content))
                    fig, ax = plt.subplots(figsize=(5, 5))
                    ax.imshow(img)
                    ax.set_title(f"Quicklook: {name}", fontsize=10)
                    ax.axis("off")
                    plt.tight_layout()
                    plt.show()
                else:
                    print(f"Failed to fetch quicklook for {name}. Status code: {response.status_code}")

        def on_index_change(change):
            update_display(change["new"])

        def go_first(_): current_index.value = 0
        def go_left(_):  current_index.value = max(current_index.value - 1, 0)
        def go_right(_): current_index.value = min(current_index.value + 1, max_index)
        def go_last(_):  current_index.value = max_index

        # Bind events
        current_index.observe(on_index_change, names="value")
        first_button.on_click(go_first)
        left_button.on_click(go_left)
        right_button.on_click(go_right)
        last_button.on_click(go_last)

        # Layout
        nav_controls = widgets.HBox(
            [first_button, left_button, current_index, max_label, right_button, last_button],
            layout=widgets.Layout(justify_content="center", align_items="center", gap="5px")
        )

        full_controls = widgets.VBox([
            output,
            nav_controls,
            play
        ], layout=widgets.Layout(align_items="center"))

        display(full_controls)
        update_display(0)
#################################################################################################################################
    def download(self):
        """
        Launch an interactive widget to download selected products as ZIP files.

        Features:
        - Displays instructions and a text input for selecting products by index.
        - Shows a progress bar that updates during downloads.
        - Downloads files to a local folder named "data_copernicus".
        - Handles expired authentication tokens by refreshing automatically.
        - Prints success or failure messages for each download.

        Requires that a product search has been performed before calling.

        No parameters or return values.
        """
        if not self.products:
            print("No products found. Run a search first.")
            return

        # Widgets
        instructions = widgets.HTML("<b>Enter indices to download:</b><br>🔹 Use comma-separated values like <i>0,2,5</i><br>🔹 Use range values like <i>0-5</i><br>🔹 Use mixe values like <i>0,3-5,8</i><br>🔹 Type <i>all</i> to download all results")
        input_box = widgets.Text(placeholder="e.g. 0,2,4 or all")
        download_button = widgets.Button(description="Download", button_style='success')
        progress = widgets.IntProgress(value=0, min=0, max=1, description='Progress:', bar_style='info')
        output = widgets.Output()

        # Display
        display(widgets.VBox([instructions, input_box, download_button, progress, output]))

        # Folder
        download_folder = Path("data_copernicus")
        download_folder.mkdir(exist_ok=True)

        def download_product(product_id, name):
            url = f"https://download.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
            headers = {"Authorization": f"Bearer {self.token}"}
            response = requests.get(url, headers=headers, stream=True)

            # Handle expired token
            if response.status_code == 401:
                self.refresh_token()
                headers = {"Authorization": f"Bearer {self.token}"}
                response = requests.get(url, headers=headers, stream=True)

            file_path = download_folder / f"{name}.zip"
            if response.status_code == 200:
                with open(file_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                return True, f"{name} downloaded."
            else:
                return False, f"Failed to download {name} – Code {response.status_code}"

        def parse_indices(input_str):
            result = set()
            parts = input_str.split(",")
            for part in parts:
                part = part.strip()
                if "-" in part:
                    start, end = part.split("-")
                    result.update(range(int(start), int(end) + 1))
                else:
                    result.add(int(part))
            return sorted(i for i in result if 0 <= i < len(self.products))

        def on_download_clicked(_):
            with output:
                clear_output()
                print("Download launched... please wait.\n")

                user_input = input_box.value.strip()
                if not user_input:
                    print("Please enter indices.")
                    return

                # Get selected products
                try:
                    if user_input.lower() == "all":
                        selected = self.products
                    else:
                        indices = parse_indices(user_input)
                        selected = [self.products[i] for i in indices]
                except Exception as e:
                    print(f"Invalid input: {e}")
                    return

                if not selected:
                    print("No valid products selected.")
                    return

                # Start download
                progress.max = len(selected)
                progress.value = 0
                progress.bar_style = 'info'

                for prod in selected:
                    success, message = download_product(prod["Product ID"], prod["Name"])
                    progress.value += 1
                    print(message)

                progress.bar_style = 'success'
                print(f"\n All downloads complete. You can find them in {download_folder}")

        download_button.on_click(on_download_clicked)

--- 

## Use of the class

First, we initialize the class.

In [4]:
client = Copernicus_API()

 Token generated


Second, we can define the parameters we are looking for. The parameters need to be written in json format. Some examples are written in this code but I am not 100% sure for the majority of the argument. To check, you can find them in the [documentation](https://documentation.dataspace.copernicus.eu/APIs/OData.html).

In [5]:
params = {
    "$filter": (
        "Collection/Name eq 'SENTINEL-2' and "
        "ContentDate/Start ge 2019-03-01T00:00:00.000Z and "
        "ContentDate/End le 2019-03-30T00:00:00.000Z and "
        "Online eq true and "
        "Attributes/OData.CSC.StringAttribute/any(att: att/Name eq 'tileId' and att/Value eq '59GNM') and "
        "Attributes/OData.CSC.DoubleAttribute/any(att: att/Name eq 'cloudCover' and att/Value lt 10) and "#less than 10% of cloud cover
        "Attributes/OData.CSC.StringAttribute/any(att: att/Name eq 'productType' and att/Value eq 'S2MSI2A')"#S2MSI1C nv L1C et S2MSI2A nv L2A
        #"OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((-5 40, -5 41, -4 41, -4 40, -5 40))')"
        #f"{filter_odata}"
        
        # "Name eq 'S2A_MSIL1C_20240501T000731_N0510_R073_T56MRV_20240501T010755.SAFE' and "
        # "Id eq 'a7527c07-4b09-465c-8e51-7e036541db4b' and "
        # "PublicationDate ge 2024-05-01T00:00:00.000Z and "
        # "PlatformSerialIdentifier eq 'Sentinel-2A' and "
        # "S3Path eq '/eodata/Sentinel-2/MSI/L1C/2024/05/01/S2A_MSIL1C_20240501T000731_N0510_R073_T56MRV_20240501T010755.SAFE' and "
        # "OrbitDirection eq 'ASCENDING' and "
        # "OrbitNumber eq 12345 and "        
        # "EvictionDate ge 2024-12-31T00:00:00.000Z"
    ),
    "$top": 20,
    "$expand": "Assets"
}

client.search_products(params)

Now we can display the searched dataframe.

In [6]:
client.display_df()

Output()

Output()

Depending on the data, Copernicus provide a Quicklook. So we can check our image before downloading them.

In [7]:
client.quicklooks()

Finally, the last calling allow an interactive donwloading depending the indice of the dataframe.

In [8]:
client.download()